# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


In [2]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['timestamp'] = pd.to_datetime(X_copy['timestamp'])
        X_copy['hour'] = X_copy['timestamp'].dt.hour
        X_copy['dayofweek'] = X_copy['timestamp'].dt.dayofweek
        X_copy.drop(columns='timestamp', inplace=True)
        return X_copy

2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.

In [3]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_col):
        self.target_col = target_col
        self.encoder = None
        self.cat_cols = None
        self._y = None

    def fit(self, X, y=None):
        self.cat_cols = [
            col for col in X.columns
            if X[col].dtype.name in ['object', 'category'] and col != self.target_col
        ]
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.encoder.fit(X[self.cat_cols])
        return self

    def transform(self, X):
        X_copy = X.copy()
        self._y = X_copy[self.target_col]
        X_copy = X_copy.drop(columns=self.target_col)

        encoded_array = self.encoder.transform(X_copy[self.cat_cols])
        df_encoded = pd.DataFrame(
            encoded_array,
            index=X_copy.index,
            columns=self.encoder.get_feature_names_out(self.cat_cols)
        )
        X_copy.drop(columns=self.cat_cols, inplace=True)
        X_final = pd.concat([X_copy, df_encoded], axis=1)

        return X_final

    def get_target(self):
        return self._y

3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).

In [4]:
class TrainValidationTest:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def split_data(self):
        X_train_full, X_test, y_train_full, y_test = train_test_split(
            self.X, self.y,
            random_state=21,
            test_size=0.2,
            stratify=self.y
        )

        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train_full, y_train_full,
            random_state=21,
            test_size=0.2,
            stratify=y_train_full
        )

        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [5]:
class ModelSelection():
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.model_names = grid_dict
        self.best_models_info = []
    
    def choose(self, X_train, y_train, X_valid, y_valid):
        best_score = 0
        best_model_name = None

        for i, grid in enumerate(self.grids):
            model_name = self.model_names[i]
            print(f"Estimator: {model_name}")

            grid.fit(X_train, y_train)
            total_combinations = len(grid.cv_results_['params'])
            tqdm_bar = tqdm(total=total_combinations)
            tqdm_bar.update(total_combinations)
            tqdm_bar.close()

            best_params = grid.best_params_
            print(f"Best params: {best_params}")

            best_estimator = grid.best_estimator_
            train_acc = accuracy_score(y_train, best_estimator.predict(X_train))
            valid_acc = accuracy_score(y_valid, best_estimator.predict(X_valid))
            print(f"Best training accuracy: {train_acc:.3f}")
            print(f"Validation set accuracy score for best params: {valid_acc:.3f}\n")

            if valid_acc > best_score:
                best_score = valid_acc
                best_model_name = model_name

            self.best_models_info.append({
                "model": model_name,
                "params": best_params,
                "valid_score": valid_acc
            })

        print(f"Classifier with best validation set accuracy: {best_model_name}")
        return best_model_name

    
    def best_results(self):
        return pd.DataFrame(self.best_models_info)

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [6]:
class Finalize():
    def __init__(self, estimator):
        self.estimator = estimator
    
    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        acc = accuracy_score(y_test, self.estimator.predict(X_test))
        print(f"Accuracy of the final model is {acc}")

    def save_model(self, path):
        joblib.dump(self.estimator, path+'.sav')
        print("Model was successfully saved")

## 4. Main program

1. Load the data from the file (****name of file****).

In [7]:
df = pd.read_csv("../data/checker_submits.csv")

2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```

In [8]:
preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('dayofweek'))
])

3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```

In [9]:
X = preprocessing.fit_transform(df)
y = preprocessing.named_steps['onehot_encoder'].get_target()

4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.

In [10]:
splitter = TrainValidationTest(X=X, y=y)
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split_data()

5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.

In [11]:
svm = SVC(probability=True, random_state=21)
svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 
               'C':[0.01, 0.1, 1, 1.5, 5, 10], 
               'gamma': ['scale', 'auto'], 
               'class_weight':('balanced', None), 
               'random_state':[21], 
               'probability':[True]}]
gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)

tree = DecisionTreeClassifier(random_state=21)
tree_params = [{'max_depth': list(range(1, 50)),
                'class_weight': ['balanced', None],
                'criterion': ['entropy', 'gini']}]
gs_tree = GridSearchCV(estimator=tree, param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)

rf = RandomForestClassifier(random_state=21)
rf_params = [{'n_estimators': [5, 10, 50, 100],
              'max_depth': list(range(1, 50)),
              'class_weight': ['balanced', None],
              'criterion': ['entropy', 'gini']}]
gs_rf = GridSearchCV(estimator=rf, param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: "SVM", 1: "Decision Tree", 2: "Random Forest"}

In [12]:
selector = ModelSelection(grids, grid_dict)
best_name = selector.choose(X_train, y_train, X_valid, y_valid)

Estimator: SVM


  0%|          | 0/72 [00:00<?, ?it/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.952
Validation set accuracy score for best params: 0.878

Estimator: Decision Tree


  0%|          | 0/196 [00:00<?, ?it/s]

Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}
Best training accuracy: 0.997
Validation set accuracy score for best params: 0.867

Estimator: Random Forest


  0%|          | 0/784 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 22, 'n_estimators': 50}
Best training accuracy: 1.000
Validation set accuracy score for best params: 0.904

Classifier with best validation set accuracy: Random Forest


In [13]:
results_df = selector.best_results()
results_df

,model,params,valid_score
0,SVM,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.877778
1,Decision Tree,"{'class_weight': 'balanced', 'criterion': 'gin...",0.866667
2,Random Forest,"{'class_weight': None, 'criterion': 'gini', 'm...",0.903704


6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

In [14]:
best_params = results_df[results_df['model'] == best_name]['params'].values[0]

if best_name == "SVM":
    model = SVC(**best_params)
elif best_name == "Decision Tree":
    model = DecisionTreeClassifier(**best_params)
elif best_name == "Random Forest":
    model = RandomForestClassifier(**best_params)
else:
    raise ValueError("Unknown model")

final = Finalize(model)
final.final_score(X_train, y_train, X_test, y_test)
model.fit(X_train, y_train)
acc = accuracy_score(y_test, model.predict(X_test))
final.save_model(f"{best_name}_{acc}")

Accuracy of the final model is 0.9053254437869822
Model was successfully saved


That is it, congrats!